In [1]:
import requests
import pandas as pd

In [2]:
# Verify if API is ON

result_on = requests.get('http://localhost:5000/')
result_on.text

'{"message":"API available","status":"success"}\n'

In [3]:
#Load insurance dataset
df = pd.read_csv('insurance.csv')

In [4]:
# Analysing columns
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [6]:
# PREPARE Endpoint
df_json = df.to_json()
post_data = {'max_na': 0.2, 'max_var': 0.2, 'payload': df_json}
url = 'http://localhost:5000/prepare'

result_prepare = requests.post(url, data=post_data)
df_prep = pd.read_json(result_prepare.text)


In [7]:
df_prep.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [8]:
df_prep.dtypes

age           int64
sex           int64
bmi         float64
children      int64
smoker        int64
region        int64
charges     float64
dtype: object

In [9]:
# TRAIN_TEST Endpoint
url = 'http://localhost:5000/train_test'
ml_method = 'Regression'
train_columns = ''
target = 'charges'
df_json = df_prep.to_json()
post_data = {'train_columns': ','.join(train_columns), 'target_column': target, 'ml_method': ml_method, 'test_size': 0.2, 'payload': df_json}
result_train_test = requests.post(url, data=post_data)
print(result_train_test.text)

{"gradient":{"MAE":2480.542602573218,"MSE":19330270.792450592,"RMSE":4396.62038302724},"linear":{"MAE":4024.4442885471926,"MSE":33518303.67739486,"RMSE":5789.499432368472},"randon_forest":{"MAE":2645.4058028428108,"MSE":21553842.939529218,"RMSE":4642.6116507338}}



In [10]:
#TRAIN Endpoint
url = 'http://localhost:5000/train_model'
post_data = {'train_columns': ','.join(train_columns), 'target_column': target, 'ml_method': ml_method, 'model_name': 'gradient', 'payload': df_json}
result_train = requests.post(url, data=post_data)
print(result_train.text)

{"file_name":"models_registry/2022-01-02-23-03-07/gradient_regressor.pickle","file_name_param":"models_registry/2022-01-02-23-03-07/gradient_regressor.json"}



In [11]:
#PREDICT Endpoint
df_predict = df_prep.drop('charges', axis=1)
df_json = df_predict.to_json()
model_path = result_train.text.split(',')[0].split(':')[1].replace('"','').replace(' ','')
url = 'http://localhost:5000/predict'
post_data = {'model_path': model_path, 'payload': df_json}

result_predict = requests.post(url, data=post_data)
print(result_predict.text)

{"prediction":{"0":18901.8320949581,"1":3825.9736309692,"2":7069.6082778968,"3":5774.0036903495,"4":4146.1651272165,"5":3968.8316164354,"6":9081.8028866612,"7":8114.1121476023,"8":8374.8243898319,"9":14437.6667126611,"10":4126.249500147,"11":27695.8419393727,"12":3185.5202897072,"13":12016.5843515329,"14":38624.1382178852,"15":3022.912162457,"16":13192.1698339923,"17":4121.8200731885,"18":11620.0530839011,"19":38511.8988165249,"20":15022.5530187805,"21":5111.2724117775,"22":2406.2484878092,"23":38692.9784025254,"24":7629.6637119633,"25":16070.4314345638,"26":14571.9959365137,"27":14096.2046145021,"28":4821.8886899606,"29":40010.6443349656,"30":36457.5174348697,"31":3016.2132573092,"32":6125.3058418702,"33":14199.7308080692,"34":40318.2118843559,"35":1547.0422882968,"36":17095.5710923479,"37":2099.3213790214,"38":42407.270512378,"39":47333.5800332416,"40":5029.5814249116,"41":6501.2398727259,"42":6733.1905384069,"43":7285.8221133305,"44":7893.2503057854,"45":12959.7997371956,"46":5953.2